# NaOH hotshot and PCR prep protocol

Begin by turning on oven in Durand lab at 150C.
1. place your well plate and fill falcon tube in slot A1 with 3ml of 2M NaOH
2. check offsets and dispense NaOH to wells
    - final concentration is 50mM NaOH so 12ul of 2M NaOH in 300ul wells
3. place your plate in the oven with glass cover for 15 minutes
4. fill falcon tube with 5ml of 1M Tris-HCL pH8 in slots A4 and A5 and dispense to wells
    - final concentration is 100mM Tris-HCL so 30ul of 1M Tris-HCL in 300ul wells
5. place your empty gDNA plates in the yellow clearline labware spots
6. refill all p200 tips (need a tip for every well you have)
7. Transfer DNA from wells to gDNA plates

PCR set up:
1. place PCR tubes in yellow clearline labware spots
2. make master mix in eppendorf tubes (1 tube per 96 tubes) placed in slots B1, B2, B3
    - your master mix should be 5% more than the based amount needed for a mix.
3. select number of tubes you will fill and dispense to PCR tubes

In [2]:
import opentrons.execute
import time
protocol = opentrons.execute.get_protocol_api('2.12')


metadata = {
    'protocolName': 'NaOH Lysis and PCR prep',
    'author': 'Andrew Hsiao <andrew.hsiao@univ-lyon1.fr>',
    'description': 'NaOH lysis protocol from a 300 well plate for PCR genotyping, followed by PCR prep of tubes',
    'apiLevel': '2.12'
}

# SETUP and INITIALIZE LABWARE:
# will return a single list ordered by row [A1, A2...A20, B1, B2] - 'left to right, top to bottom'
def get_wells_by_row(plate):
    rows = plate.rows()  # list of list ordered by row [[A1,A2], [B1,B2]]
    wells_by_row = []
    for i in range(len(rows)):
        for j in range(len(rows[i])):
            wells_by_row.append(rows[i][j])
    return wells_by_row

# load tips--> note p200 tips are used here
tiprack_200_10 = protocol.load_labware('fisher_96_tiprack_200ul', '10')
tiprack_200_11 = protocol.load_labware('fisher_96_tiprack_200ul', '11') 
tiprack_200_9 = protocol.load_labware('fisher_96_tiprack_200ul', '9')

# load labware
eppendorfs = protocol.load_labware('randlettlab_combirack', 1)
plate_gDNA_8 = protocol.load_labware('clearlineinyellowpurpleclearline_96_wellplate_330ul', '8')
plate_gDNA_7 = protocol.load_labware('clearlineinyellowpurpleclearline_96_wellplate_330ul', '7')
plate_gDNA_4 = protocol.load_labware('clearlineinyellowpurpleclearline_96_wellplate_330ul', '4')
plate_300well = protocol.load_labware('randlettlab_300_wellplate_300ul', '2')
# getting list of wells - 'left to right, top to bottom'
wells = get_wells_by_row(plate_300well)
# combine all gDNA plates into one list
gdna_plate_wells = plate_gDNA_7.wells() + plate_gDNA_4.wells() + plate_gDNA_8.wells() # len = 288

# load pipettes
# p1000 = protocol.load_instrument('p1000_single_gen2', 'left', tip_racks=[tiprack_1000_9])
p300 = protocol.load_instrument('p300_single_gen2', 'right', tip_racks=[tiprack_200_10, tiprack_200_11, tiprack_200_9])

protocol.home()
print('done homing, run next cell')
# how to delete labware
#del protocol.deck['1'] # this deletes what is in slot 1, and can load something else

# slack messenger bot
def send_slack_message(message: str):
    import requests
    
    payload = '{"text": "%s"}' % message 
    response = requests.post(
        "SLACK ACCOUNT HOOK HERE", 
        data = payload
        )
    print(response.text)
print('registered slack messenger')

done homing, run next cell
registered slack messenger


# Set offsets and check positions for p300 pipette (using p200 tips)

In [3]:
# reset offsets for p300 USING p200 TIPS
tiprack_200_10.set_offset(x=-3.8, y=2, z=-2)
tiprack_200_11.set_offset(x=-3.8, y=2, z=-2)
tiprack_200_9.set_offset(x=-3.8, y=2, z=-2)
plate_gDNA_8.set_offset(x=-3.5, y=2.2, z=-3)
plate_gDNA_7.set_offset(x=-3.5, y=2.2, z=-3)
plate_gDNA_4.set_offset(x=-3.5, y=2.2, z=-3)
eppendorfs.set_offset(x=-0.2, y=0.2, z=0)
# plate_300well.set_offset(x=.9, y=-2, z=-6)

# check the location of things
def check_positions(check_pipette, labware_list, well_number = 0, dwell_time = 1):    
    if check_pipette.has_tip:
        check_pipette.drop_tip()
    check_pipette.pick_up_tip()
    for labware_name in labware_list:
        well_label = labware_name.wells()[well_number]
        print(well_label)
        check_pipette.move_to(well_label.top())
        time.sleep(dwell_time)
    check_pipette.return_tip()

# checking for PCR tubes, well plate, and tip racks
check_positions(p300,[plate_gDNA_7, plate_gDNA_4, plate_gDNA_8, plate_300well, tiprack_200_11, tiprack_200_9],well_number = 0, dwell_time = 1)
check_positions(p300,[plate_gDNA_7, plate_gDNA_4, plate_gDNA_8, plate_300well, tiprack_200_11, tiprack_200_9],well_number = -1, dwell_time = 1)
# checking for eppendorf and falcon tubes
check_positions(p300,[eppendorfs],well_number = 5, dwell_time = 1)
check_positions(p300,[eppendorfs],well_number = 0, dwell_time = 1)

print('if all looks good, run next cell')

A1 of ClearLine 96 Well Plate 330 µL - Domi Domi on 7
A1 of ClearLine 96 Well Plate 330 µL - Domi Domi on 4
A1 of ClearLine 96 Well Plate 330 µL - Domi Domi on 8
A1 of AAA Randlett Lab 300 Well Plate 300 µL on 2
A1 of Fisher 96 Tip Rack 200 µL on 11
A1 of Fisher 96 Tip Rack 200 µL on 9
H12 of ClearLine 96 Well Plate 330 µL - Domi Domi on 7
H12 of ClearLine 96 Well Plate 330 µL - Domi Domi on 4
H12 of ClearLine 96 Well Plate 330 µL - Domi Domi on 8
O20 of AAA Randlett Lab 300 Well Plate 300 µL on 2
H12 of Fisher 96 Tip Rack 200 µL on 11
H12 of Fisher 96 Tip Rack 200 µL on 9
B1 of AA combi rack 15ml 1.5ml 2ml on 1
A1 of AA combi rack 15ml 1.5ml 2ml on 1
if all looks good, run next cell


# How many fish do you have?
change the variable below and run the cell

In [15]:
num_fish = 288 # change here and dont forget to run this cell!

print("cell has been run and you have " + str(num_fish) +" fish. Continue!")

cell has been run and you have 288 fish. Continue!


# Distribute NaOH to well plate

- add no more than 3 ml of 2M NaOH to falcon tube and place in slot A1 of eppendorf rack (top left).

In [14]:
#reset pipettes to first tip in tiprack (because of offset calibrate step)
p300.reset_tipracks()
# set stats
p300.well_bottom_clearance.aspirate = 2
p300.well_bottom_clearance.dispense = 2.5

def distribute_naoh(num_wells):
    p300.distribute(7.5, eppendorfs['A1'], wells[:num_wells], new_tip='once', blow_out=True, blowout_location='source well', disposal_volume = 10)
    print("all done! oven time!! 15 minutes please :)")

distribute_naoh(num_fish) # 6 min for 288 wells
send_slack_message("all done with NaOH! - domi domi")

all done! oven time!! 15 minutes please :)
ok


# Distribute Tris to well plate

- add no more than 5 ml of 1M Tris pH8 to 2 falcon tubes and place in slot A4 and A5 of eppendorf rack (top right).

In [16]:
# set stats
p300.well_bottom_clearance.aspirate = 2
p300.well_bottom_clearance.dispense = 4.5

gdna_plate_wells = plate_gDNA_7.wells() + plate_gDNA_4.wells() + plate_gDNA_8.wells() # len = 288

def distribute_tris_1(start,end,tube):
    p300.distribute(30, eppendorfs[tube], wells[start:end], new_tip='once', blow_out=True, blowout_location='source well', disposal_volume = 10)

if num_fish <= 144:
    distribute_tris_1(0,num_fish, 'A4')
else:
    distribute_tris_1(0,144, 'A4')
    distribute_tris_1(144,num_fish, 'A5')
print("all done! set up to transfer your DNA to gDNA plates now :)")
send_slack_message("all done with Tris! - domi domi")

all done! set up to transfer your DNA to gDNA plates now :)
all done! set up to transfer your DNA to gDNA plates now :)
ok


# Transfer DNA from well plate to gDNA plates

- place 3 p200 tipracks in lab slots 9, 10, 11
- place gDNA plates in lab slots 4, 8, 7

In [ ]:
p300.well_bottom_clearance.aspirate = 3
p300.well_bottom_clearance.dispense = 6

p300.reset_tipracks()

# transfer fish DNA from well plate to gDNA plates
def transfer_fishies(pipette, volume, source_wells, target_wells, mix_before = False, mix_vol = 150, mix_times = 5):
    # transfer from source wells to target wells, no homing in between
    n_wells = len(source_wells)

    for i in range(n_wells):
        pipette.pick_up_tip()

        if mix_before:
            for rep in range(mix_times):
                pipette.aspirate(mix_vol, source_wells[i]) 
                pipette.dispense(mix_vol, source_wells[i])   
            pipette.blow_out(source_wells[i])

        pipette.aspirate(volume, source_wells[i] )
        pipette.dispense(volume, target_wells[i])
        pipette.drop_tip(home_after=False)
    print('Done! Your gDNA plates are ready for use :)')
        
transfer_fishies(p300, 100, wells[:num_fish], gdna_plate_wells, mix_before=True, mix_vol = 100, mix_times = 1)

send_slack_message("all done transferring DNA to gDNA plates! - domi domi")

# Distribute master mix to PCR tubes

- add eppendorfs in first 3 slots filled with your master mix

For 96 tubes (5% more than base mix):
        - 201.6 taq
        - 10.1 FR primer mix
        - 594.7 H2O

In [28]:
#reset pipettes to first tip in tiprack (because of offset calibrate step)
p300.reset_tipracks()
# set stats
p300.well_bottom_clearance.aspirate = 1
p300.well_bottom_clearance.dispense = .4

# distribute master mix to PCR tubes
def dispense_master_mix(num_pcr_plates):
    if 1 <= num_pcr_plates <= 3:
        print("dispensing master mix to " + str(num_pcr_plates) + " plate/s...")
    else:
        print("woah woah woah...we've only got space for up to 3 plates")
        return   
    for x in range(num_pcr_plates):
        tube_count ='B' + str(x + 1)
        p300.distribute(7.9, 
                eppendorfs[tube_count], 
                gdna_plate_wells[x*96:(x+1)*96], 
                new_tip='once',
                touch_tip = False,
                blow_out=True, 
                blowout_location='source well',
                disposal_volume = 6)
    print("all done! add your DNA now :)")
    
# fyi - logic for function       
# 0 b1   0:96
# 1 b2   96:192
# 2 b3   192:288

# Master mix should be 5% more so for 96 tubes --> 100.8 tubes
# taq: 201.6
# primer mix: 10.1
# H2O: 594.7
# one eppendorf per 96 PCR tubes


dispense_master_mix(3) # about 8 min for 3 plates

send_slack_message("your master mix is ready! - domi domi")

dispensing master mix to 3 plate/s...
all done! add your DNA now :)
